# Coleta dos nomes de usuário dos jogadores

### Antes de podermos coletar nossos dados precisamos importar os módulos que iremos utilizar

- multiprocessing: Será utilizado para paralelizar nossos processos
- selenium: Irá simular um usuário navegando pela internet coletando os dados
- os: Irá criar as pastas caso elas não existam para manter uma hierarquia de arquivos organizada
- pathlib: Utilizar caminhos não relativos até arquivos

In [ ]:
from multiprocessing import Pool
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import *
import os
import pathlib

### Vamos inicializar algumas variáveis globais para poupar certo tempo depois

In [ ]:
url = 'https://www.mobafire.com/league-of-legends/players?sort_type=create_ts&sort_order=desc&name=&'

options = Options()
options.add_argument('--headless')
driver_path = pathlib.Path(__file__).parent / 'chromedriver.exe'


### Criamos aqui a classe que fará toda a mágica de recuperar os nomes de usuário para nossa aplicação

#### Um breve passo a passo do seu funcionamento
1. Recupera o endereço que irá simular a função de filtro do site (o filtro era utilizado pela URL, então nada melhor do que simular o clique mandando direto a URL)
2. Abrimos a página com o filtro e tentamos encontrar um elemento que sinaliza o fim dos registros, ou seja, se chegamos na última página um texto era exibido, se encontramos esse texto paramos
3. Recuperamos a tabela que contém o nome dos jogadores e pegamos todas as suas linhas
4. Iteramos sobre as linhas recuperando o nome do jogador somente, ignorando qualquer outra informação disponível naquela linha
5. Criamos a estrutura de diretórios para o servidor especificado e o elo especificado
6. Adicionamos os registros recuperados nessa página e continuamos o loop

In [ ]:
def get_users_nick(server, elo, driver):
    filt = f'server={str(server).upper()}&elo_min={str(elo).title()}&elo_max={str(elo).title()}'
    count = 1

    while 1:
        driver.get(url + filt + f'&page={count}')

        try:
            _ = driver.find_element_by_xpath('//*[@id="browse-players"]/p').text
            break
        except NoSuchElementException:
            pass

        table = driver.find_element_by_xpath('//*[@id="browse-players"]/div[2]/div/table/tbody')
        rows = table.find_elements_by_tag_name('tr')

        nicknames = []
        for element in rows:
            nicknames.append(str(element.find_elements_by_tag_name('td')[1].find_element_by_tag_name('a').text))

        file = f'{server}'
        os.makedirs(file, exist_ok=True)

        with open(f'{file}/{elo}.txt', 'a+') as writable:
            writable.write('\n'.join(nicknames))

        count += 1


### Como toda a mágina ocorre no método acima, esse método acaba sendo um pouco ignorado, mas ele é de grande importância, pois ele possibilita o paralelismo do nosso programa

#### Como ele faz isso? Bom, vejamos
1. Ele cria uma lsita com todos os elos possíveis
2. Ele cria um driver só para ele, gerando assim múltiplos drivers ao mesmo tempo
2. Para cada elo ele chama o método da mágica contendo o servidor que lhe foi passado
3. Fecha o driver e apaga a variável da memória para evitar quaisquer problemas

In [ ]:
def call_for_each_server(server):
    elos = ['bronze', 'silver', 'gold', 'platinum', 'diamond', 'master', 'challenger']

    driver = Chrome(str(driver_path.absolute()), chrome_options=options)
    for elo in elos:
        get_users_nick(server, elo, driver)

    driver.close()
    driver.quit()
    del driver


### A célula abaixo apenas envia todos os servidores para diferentes processos do sistema que irão executar o método "call_for_each_server" em paralelo

In [ ]:
servers = ['br', 'eune', 'euw', 'kr', 'lan', 'las', 'na', 'oce', 'ru', 'tr', 'ch', 'ph', 'sea']

with Pool(processes=13) as p:
    p.map(call_for_each_server, servers)

<p>_____________________________________________________________________________________________________________________________<p>

# Coleta das estatísticas dos jogadores

### Vamos fazer alguns imports importantes para essa etapa

- selenium: Novamente, mas utilizando a classe Select para tratar alguns elementos da web que não eram necessários antes
- inflect: Para imprimir verbalmente os números. 1 -> Primeiro, 2 -> Segundo, etc.
- random: Só porque eu sou noiado e desconfiei que ele não estava agregando nada aos arquivos, somente utilizando os antigos

In [ ]:
from selenium.webdriver.support.select import Select
import inflect
import random

### Definindo novamente a nossa variável global URL, pois ela foi alterada na execução deste script

In [ ]:
url = 'https://teemo.gg/player'

### Define um método auxiliar que irá ser utilizado para encontrar um elemento no HTML

In [ ]:
def try_to_find_element(driver, identificador, kind):
    import time
    start = time.time()

    identificador = identificador.replace('"', "'")
    while time.time() - start < 5:
        try:
            _ = exec(f'driver.find_element_by_{kind}("{identificador}").text')
            return True
        except NoSuchElementException:
            pass

    return False

### O método abaixo coleta os três campeões que nosso jogador mais utiliza e os retorna em formato de lista

#### Passo a passo da execução:
1. Entra na página do jogador utilizando a URL - neste site a URL também era utilizada como filtro de páginas
2. Encontra o elemento que irá ordenar a lista de campeões dos mais jogados para os menos jogados
3. Utiliza o elemento anterior para ordenar a lista
4. Itera do primeiro até o terceiro elemento coletando o nome dos campeões e desconsiderando qualquer outra informação que possa possuir no HTML do site

 Caso haja algum erro, o programa é terminado com a messagem de erro "Deu erro na mastery" para identificar a origem.

In [ ]:
def get_mastery_champions(server, player, driver):
    driver.get(f'{url}/champion-masteries/{server}/{player}')
    sort = '//*[@id="champion-mastery-sorter"]'
    dropdown = Select(driver.find_element_by_xpath(sort))
    dropdown.select_by_visible_text('Mastery Points')

    champions = []
    for i in range(1, 4):
        xpath = f'//*[@id="champ-mastery-items"]/div/div[2]/div/div[{i}]/p'
        if try_to_find_element(driver, xpath, 'xpath'):
            champions.append(driver.find_element_by_xpath(xpath).text)
        else:
            exit('Deu erro na mastery')

    return champions

### É feio, mas funciona. O método abaixo recupera todas as estatísticas do jogador

São 11 estatísticas sendo elas:
1. Games: Quantidade de jogos jogados
2. Remakes: Quantidade de partidas refeitas
3. Playing time: Quanto tempo de jogo dentro de partidas
4. Kills: Quantidade de abates em partidas
5. Deaths: Quantidade de mortes em partidas
6. Assists: Quantidade de assistência em abates em partidas
7. Gold Earned: Quantidade total de ouro ganho em partidas
8. Pentakills: Quantidade de abates quíntuplos em partidas
9. Wards Placed: Quantidade de vigias colocadas em partidas
10. Minions Killed: Quantidade de minions abatidos em partidas
11. Total Damage: Dano total causado em partidas
 
 Caso haja algum erro, o programa é terminado com a messagem de erro "Deu erro nos stats" para identificar a origem

In [ ]:
def get_player_status(server, player, driver):
    driver.get(f'{url}/statistics/{server}/{player}')
    score_board = driver.find_element_by_id('tm-scoreboard')
    if try_to_find_element(driver, 'tm-scoreboard', 'id'):
        stats = score_board.find_elements_by_class_name('points')
        return [
            stats[0].text,
            stats[1].text,
            stats[2].text,
            stats[3].text,
            stats[4].text,
            stats[5].text,
            stats[6].text,
            stats[7].text,
            stats[8].text,
            stats[9].text,
            stats[10].text
        ]
    else:
        exit('Deu erro nos stats')

### Esse método recupera a nossa coluna target dos dados e é o único que, caso não encontrado, não é adicionada a linha do jogador à tabela de dados

#### Passo a passo:
1. Entra na página do jogador com o elo e o servidor informados
2. Procura o elemento no HTML onde deveria conter o elo
3. Caso encontre pode ter um problema, isso é, se o elo for Unranked, nós ignoramos e informamos o elo que o jogador deveria ter. Se não retornamos o elo encontrado

 Caso haja algum erro, é retornado um IndexError para informar que deve ser pulada essa linha

In [ ]:
def get_player_elo(server, player, driver, elo):
    driver.get(f'{url}/resume/{server}/{player}')
    xpath = '/html/body/div/section[1]/div/div[1]/div[2]/div[2]/p[1]/b'
    if try_to_find_element(driver, xpath, 'xpath'):
        current = driver.find_element_by_xpath(xpath).text
        return elo if current == 'Unranked' else current

    else:
        raise IndexError

### O maior método entre todos, itera sobre todos os servidores recuperando as informações dos jogadores fazendo uso dos métodos explicados anteriormente. Cria a estrutura de pastas apropriada para cada jogador

#### Algumas observações importantes devem ser mencionadas:
- O tratamento de erros foi feito com cuidado para que não seja inserida alguma informação errada na tabela de dados, sendo assim, todos os jogadores da tabela de dados existe e pode ser encontrado no site "https://teemo.gg/"
- A estrutura de arquivos foi feita da seguinte forma: 
    - __nome_servidor / nome_elo.txt__ : para os arquivos com os nomes dos jogadores
    - __player_nome_servidor / file.txt__ : para os arquivos com as estatísticas dos jogadores
- O método é executa múltiplas vezes, para prevenir repetição de dados, o arquivo sempre é lido e escrito somente os jogadores não conflitantes, ou seja, os jogadores novos

In [ ]:
def iterate_through_servers(path):
    elos = ['bronze', 'silver', 'gold', 'platinum', 'diamond', 'master', 'challenger']

    if path.find('/') != '-1':
        server = path.split('\\')[-1]
    else:
        server = path.split('/')[-1]

    driver = Chrome(executable_path=str(driver_path.absolute()), options=options)
    resp = []
    for elo in elos:
        try:
            with open(f'{path}/{elo}.txt', 'r') as players:
                for player in players.read().split('\n'):
                    try:
                        actual_elo = get_player_elo(server, player, driver, elo)
                    except IndexError:
                        continue

                    status = get_player_status(server, player, driver)
                    champions = get_mastery_champions(server, player, driver)

                    resp.append((actual_elo, * status, *champions))
        except FileNotFoundError:
            pass

    path = pathlib.Path(path).parent / f'players_{server}'
    if not os.path.exists(str(path.absolute())):
        os.mkdir(str(path.absolute()))
        open(f'players_{server}/file.txt', 'a').close()

    writable_players = []
    with open(f'players_{server}/file.txt', 'r') as file:
        lines = file.read().split('\n')
        writable_players.extend(lines)
        for i in resp:
            if str(i) not in lines:
                writable_players.append(str(i))

    with open(f'players_{server}/file.txt', 'w') as file:
        file.write('\n'.join(writable_players))

    driver.close()
    driver.quit()

### Método auxiliar para lista todas as pastas que deveriam ser utilizadas para recuperação das estatísticas

Ele percorre as pastas no diretório local e procura por pastas que não comecem por "player" nem "venv" e não é um arquivo executável nem script, sobrando somente os arquivos com os nomes dos jogadores

In [ ]:
def get_folders(path):
    dirs = []
    for directory in os.listdir(str(path.absolute())):
        if directory != 'venv' and not directory.endswith('exe') and not directory.endswith(
                'py') and directory != '.idea' and not directory.startswith('player'):
            dirs.append(str((path / directory).absolute()))

    return dirs

### Esse método possibilita a divisão do processamento para cada pasta encontrada, ou seja, a pasta dos jogadores BR será executada num processo diferente da pasta dos jogadores EUW

In [ ]:
def split_servers():
    folders = random.shuffle(get_folders(pathlib.Path()))

    with Pool(processes=12) as p:
        p.map(func=iterate_through_servers, iterable=folders)

### Realiza todo o processo 10 vezes exibindo em qual iteração ele está de forma bonita

In [ ]:
eng = inflect.engine()
for i in range(1, 11):
    print(f'Started {eng.number_to_words(eng.ordinal(i))} collection, trying again...')
    split_servers()

<p>_____________________________________________________________________________________________________________________________<p>

# Reunindo todas as informações

### Primeiro precisamos recuperar todos os arquivos que começam com "player" como foi definido pela nossa estrutura

In [ ]:
def get_folders(path):
    dirs = []
    for directory in os.listdir(str(path.absolute())):
        if directory.startswith('player'):
            dirs.append(str((path / directory).absolute()))

    return dirs

root = pathlib.Path(__file__).parent

### Utilizaremos um método de auxílio que irá ler o arquivo csv e retornar todo seu conteúdo

In [ ]:
def read_csv(path):
    with open(path + '/file.txt', 'r') as csv_file:
        return csv_file.read()

### Concatenamos todos os arquivos lidos no mesmo arquivo nomeado "dataset.csv"

In [ ]:
def concatenate_files(folders):
    with open('dataset.csv', 'a') as dataset:
        for f in folders:
            csv_file = read_csv(f)
            dataset.write(csv_file)

### Chamamos o método que realiza a concatenação e é só partir para o abraço

In [ ]:
concatenate_files(get_folders(root))